# Data Dependencies in the Master Mind CodeMaker Scoring Algorithm 

In [ ]:
# set this notebook to use a large part of the browser window width
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:70% !important; }</style>"))

## Introduction

We consider that the game of Master Mind consists of a CodeMaker player 
and a CodeBreaker player. 

The CodeMaker first comes up with a number 'secret' of N digits
(we restrict each digit to be between 1 and N). 

Then the CodeBreaker comes up with guesses 'guess' for that number,
where the CodeMaker then, for each guess by the CodeBreaker, 
judges how many digits are on the right place between 'guess' and 'secret'.
From this, the CodeBreaker learns and improves his guess until guess = secret.

This Python3 Notebook tests, in a high level language, the code to solve the 
CodeMakers part of Master Mind. We do this in two iterations,
an obvious approach (where we need two sequential loops to derived the two numbers),
and then an optimised approach, leading to more compact code by combining the loops in one loop.

For old times' sake, 
both are then converted into low level assembly code for the HP-15C and HP-15C 11 calculator. 

In [ ]:
# Some introductory initialisations are needed.

prefix = 'HP-15C display'
n_display_chars = 10

secret = '4323'  # generated by HP-15C and stored in R0, then separated into digits into R1 to RN
N = len(secret)  # store in HP-15C R0, this is a user input in the HP-15C program
print('N = {:d}'.format(N))
s = str(N)
print('  >>> ' + prefix + 'I_: [{:s},'.format(s) + \
      (' '*(n_display_chars-len(s)) + '] <<<')) # not counting ',' as a position

assert int(N) == N
assert N >= 1
assert N <= 6  # This is a restriction coming 
# From the display of the HP-15C which has 10 digits only, we could only maximally store N=8 digits, 
# so that we can suffix it with a comma (,) which takes no extra digit space on the display,
# and then still have two digits to show the number of correclty [positioned digits
# and the number of wrongly positioned 'right' digits.
# However, because we need some extra register variables, we must restrict N to 6 in the end. 
# We think that's hard enough for a human player already. :)

matched_secret = ['0', '0', '0', '0']  # put in R21 to R(20+N)
matched_guess  = ['0', '0', '0', '0']  # put in R31 to R(30+N), so 37 is the highest register needed,
# since N <= 7.

verbose = 0

print('secret = ' + secret)

## Straightforward Approach with Two Sequential Loops

The functions calculate_n_correctly_positioned_digits and
calculate_n_displaced_digits who are doing what they say
since they are aptly named. :)

Note that it is important to execute calculate_n_displaced_digits
first, before calculate_n_displaced_digits is executed.

The reason is that the correctly positioned pegs/digits need to be marked as already considered, (in the matched_secret, matched_guess arrays that track of that information) to avoid that digits (both in secret and in guess) which have been matched already will be compared and potentially counted again. Each digit can only be counted at most once (when a match is found).
It's not just that there are two categories, correctly and wrongly positioned digits, but the correctly positioned ones really need to be treated first.

If in calculate_n_correctly_positioned_digits, a digit on position j in the secret 
is the same as the digit in position k of the guess,
neither digit is again used in comparisons in the second check function calculate_n_displaced_digits,
but if they are unequal, both will still be used in the second function.

This is a data dependency that imposes hard constraints on the order between some of the comparisons to perform here. This dependency is reflected in the code by the first function potentially writing to matched_secret and matched_guess 1D arrays, which are being read by the second function.

One can argue that when the functions are swapped in execution order, that is also the case, which is true, but one would get a different, and not intended result, in a lot of cases. In other words such an order swapped algorithm would not implement the desired behaviour.

In [ ]:
# just a helper function:
def print_matched(matched_secret, matched_guess, indent=0):
    s = '  ' * indent
    if verbose >= 1:
        print(s + 'matched_secret = ' + str(matched_secret))
        print(s + 'matched_guess  = ' + str(matched_guess))

def calculate_n_correctly_positioned_digits(
    secret, guess, matched_secret, matched_guess):
    if verbose >= 1:
        print('  calculate_n_correctly_positioned_digits')
    c = 0
    for i in range(0, N):
        if guess[i] == secret[i]:
            matched_secret[i] = '1'
            matched_guess[i] = '1'
            if verbose >= 1:
                print('  [i,i],[secret[i],guess[i]] =')
                print(' [{:d}, {:d}], [{:s}, {:s}]'.\
                      format(i,i, secret[i], guess[i]))
                print_matched(matched_secret, matched_guess, 2)
            c += 1

    return c

def calculate_n_displaced_digits(
    secret, guess, matched_secret, matched_guess):
    if verbose >= 1:
        print('  calculate_n_displaced_digits')
    d = 0
    for j in range(0, N):
        for k in range(0, N):
            if matched_secret[j] == '0':
                if matched_guess[k] == '0':
                    if secret[j] == guess[k]:
                        matched_secret[j] = '1'
                        matched_guess[k]  = '1'
                        if verbose >= 1:
                            s = '    [j,k],[secret[j],guess[k]] = '
                            s += '[{:d}, {:d}], [{:s}, {:s}]'.\
                                  format(j,k, secret[j], guess[k])
                            print(s)
                            print_matched(matched_secret, matched_guess, 3)
                        d += 1
    return d


def test_counting_functions():
    turn = 1
    guesses = ['2443', '3432', '4313', '4323']
    for guess in guesses:
        # In the HP-15C, guess is directly stored in R.0 to R.N, 
        # which are R11 to R1N (where '1N' stands for R[10+N], 
        # so surely contained in R11..R17 
        print('  turn = {:d} ------------------------------'.format(turn))
        print('  guess = ' + guess)
        if verbose >= 1:
            print()

        print('  >>> ' + prefix + 'I_: [{:s},'.format(guess) + \
              (' '*(n_display_chars-N) + '] <<<')) # not counting ',' as a position

        matched_secret = ['0', '0', '0', '0']
        matched_guess  = ['0', '0', '0', '0']
        if verbose >= 1:
            print_matched(matched_secret, matched_guess, 1)
            print()

        n_correctly_positioned_digits = calculate_n_correctly_positioned_digits(
            secret, guess, matched_secret, matched_guess)
        if verbose >= 1:
            print('  n_correctly_positioned_digits = {:d}'.\
              format(n_correctly_positioned_digits))
            print()

        n_displaced_digits = \
            calculate_n_displaced_digits(
            secret, guess, matched_secret, matched_guess)
        if verbose >= 1:
            print('  n_displaced_digits = {:d}'.\
                  format(n_displaced_digits))
            print()

        assert n_correctly_positioned_digits + n_displaced_digits <= N

        s1 = '{:s},{:1d}{:1d}'.format(guess,\
            n_correctly_positioned_digits, 
            n_displaced_digits)
        if verbose >= 1:
            print(len(s1))

        s1 += ' ' * (n_display_chars - (len(s1)-1)) # not counting ',' as a position
        assert len(s1) == n_display_chars + len(',')
        print('  >>> ' + prefix + 'O1: [' + s1 + '] <<<')
        turn_str = '{:d}'.format(turn)
        suffix = ' ' * (n_display_chars - len(turn_str)) # we do not count ',' as a position
        print('  >>> ' + prefix + 'O2: [' + turn_str + ',' + suffix + '] <<<\n')

        turn += 1
        
test_counting_functions()

The output is intended to be close to what an implementation on a HP-15C calculator with a 10 digit only display would be.
The first guess would be 2443 (displayI_ for display input) , and the first answer would be 2443,12 (displayO1 for display output 1) where the ,12 indicates ,cd so the correct and displaced number of digits
and the second answer would be 1, (displayO2 for display output 2) indicating ,t where t is the number of turns the CodeBReaker already used up. 

## Refactored Approach Combining the above Two Loops into One Loop

What still feels suboptimal to the program version above is that we have one loop to deduce the digits in the right place and one, after that, to deduce the digits on the wrong place. There is some commonality of body code in those loops. So a nicer program, reusing common code, could result from trying to merge both loops into one. However, we need to maintain the data dependency discussed above to ensure algorithm correctness.

### Combining Loop Bodies
We notice some similarities in the bodies of the two above functions.
In particular:
            if matched_secret[j] == '0':
                if matched_guess[k] == '0':
which occurs in the second function, which one can consider to also add to the first function loop body,
since they always evaluate to true, since '0' is the initialisation value for 
both the matched_secret and matched_guess array.

### Ensuring Correct Order of Index-Couples
Note that it is still important to execute calculate_n_displaced_digits
first, before calculate_n_displaced_digits is executed.
So to guarantee that, we will have to execute the indices j and k on the diagonal, 
where j==k first, before executing any other index-couple.
This can be done with 'loop transformation'.
More specifically, the lines:
<pre>
    for diag in range(0, N):
        for on_diag in range(0, N):
            j = on_diag % N # say vertical
            k = (diag + on_diag) % N # say horizontal
</pre>
below ensure this index-couple order.
The added assert statements check at run time that no diagonal index couple
is visited after any non-diagonal index-couple has just been visited.
It's clear that all index-couples are visited and from those two claims, 
also that all diagonal index-couples are tested before any non-diagonal index pair is tested.

As for loop transformations like this, I read about it in 1991 from a book
of Utpal Banerjee [1],[2], I obtained from the IMEC library as a student. They are very useful
for compilers, both if you want to allow the compiler to restructure the code for 
efficiency in terms of reducing the number of lines. For this, dependency analysis
in terms of data flow is important. But also a parallellizing compiler, targetting
not one but multiple processing units, can, when they understand this data dependency,
derive what operations can be executed in parallel (since they are not interdependent)
and which ones cannot (since they have a data dependency and so should be executed 
sequentially). I remember having this epiphany while reading Utpal Banerjee's book on this
and expecially liked the automatic procedure in finding these optimising transformations.
Later, on my MSc in Computation at Oxford University in 1995, 
I took a course in Bulk Synchronous Parallellism (BSP), 
where it was again one of the major 
techniques co-invented/discovered by Oxford's Bill McColl in 1992 [3] in obtaining efficient 
parallellisation, essentially auto-discovering data-dependencies from an even purely seuqntial
implementation and from that seeing opportunities for parallallisation (wherever there is no data dependency)
and then following the data flow with a 'barrier' of parallel processors.

In our case of Master Mind, say we consider the j (4 secret digits) as the vertical axis 
and the k (4 guess digits) as the horizontal one.
Then in step 1, the barrier is a line that starts on the diagonal (where in principle the first function could be executed in parallel on the 4 digits, so say with 4 processors at the same time).

In step 2, that line, with the same diagonal slope, is moved right one position and the second function could be executed in parallel again
by 4 parallel processors simultaneously. Then in step 3, as well as 4, the line moves right again.
In fact, steps 2,3,4 over the 3 * 4 = 12 matrix elements could be executed entirely in parallel, since there are no data dependencies.  That is not to say that the same index-pairs would be visited then (in some
schemes more index pairs would be skipped than in others), but the outcome answer: the number of displaced digits (d) would be the same.

### Optimised Implementation
The code below implements both the above requirements, merging into one loop and retaining data dependencies,  together. Note that the code under the check_index_order branch is just for demonstration purposes and can be left out in an actual implementation.

In [ ]:
secret = '4323'
guess = '4313'
verbose = 2

def calculate_all_digits(secret, guess, check_index_order=True):
    matched_secret = ['0', '0', '0', '0']
    matched_guess  = ['0', '0', '0', '0']

    if verbose >= 1:
        print('  calculate_all_digits')
    c = 0
    d = 0
    branch = 0
    for diag in range(0, N):
        for on_diag in range(0, N):
            j = on_diag % N # say vertical
            k = (diag + on_diag) % N # say horizontal
            if check_index_order:
                if j==k:
                    assert j==k
                    assert branch in [0,1]  # all occurences of 
                    # this branch 1 happen before any of branch 2,
                    # so cannot come in here from branch 2.
                    if branch == 0:
                        branch = 1
                    assert branch == 1 
                else:
                    assert j!=k
                    assert branch in [1,2]  # all occurences of 
                    # this branch 2 happen after any of branch 1
                    if branch == 1:
                        assert j==0
                        assert k==1
                        branch = 2
                    assert branch == 2
                
            if verbose >= 1:
                print('  (j,k) = ({:d},{:d})'.format(j,k))
            if matched_secret[j] == '0':
                if matched_guess[k] == '0':
                    if secret[j] == guess[k]:
                        matched_secret[j] = '1'
                        matched_guess[k]  = '1'
                        if verbose >= 2:
                            s = '    [j,k],[secret[j],guess[k]] = '
                            s += '[{:d}, {:d}], [{:s}, {:s}]'.\
                                  format(j, k, secret[j], guess[k])
                            print(s)
                            print_matched(matched_secret, matched_guess, 3)
                        if j==k:
                            c += 1
                        else:
                            d += 1
    
    print('--(c,d) = ({:d},{:d})-\n'.format(c,d))                
    return c,d

We now run the second approach and check that for just one guess comparison to the secret,
all index couples are visited and
in the right order and that the result is the same as for the above initial code.

In [ ]:
verbose = 2
calculate_all_digits(secret, guess)

So this is all correct.

We now test the same function on the 4 guesses above, this time non verbose, as we know the index-couple traversal order is checked already and not changing here.

In [ ]:
def test_single_counting_function():
    turn = 1
    guesses = ['2443', '3432', '4313', '4323']
    cds = [(1,2), (0,4),(3,0),(4,0)]
    for guess in guesses:
        print('  guess = ' + guess)
        matched_secret = ['0', '0', '0', '0']
        matched_guess  = ['0', '0', '0', '0']
        c,d = calculate_all_digits(secret, guess)
        assert (c,d) == cds[turn-1]
        turn += 1

print('secret = {:s}\n'.format(secret))        
verbose = 0
test_single_counting_function()

So, those are the correct answers. :)

# References

[1] Utpal Banerjee. Dependence Analysis (Loop Transformation for Restructuring Compilers). Springer; 1 edition (October 31, 1996).
You can find an index of the book at https://link.springer.com/content/pdf/bfm%3A978-0-585-28122-3%2F1.pdf

[2] Utpal Banerjee. Loop parallelization. Kluwer Academic Publisher, 1994.

[3] Bulk-Synchronous Parallellism https://en.wikipedia.org/wiki/Bulk_synchronous_parallel


This code is available at https://github.com/PeterSels/OnComputation/tree/master/DataDependencies

Peter Sels, April 28th 2020. Dedicated to Utpal.

Copyright © 2020 Logically Yours BV.